In [1]:
# python version
!python --version

Python 3.7.16


- q: 종료
- s: 이미지 저장
- y: YOLOv5 실행
- p: YOLOv5 결과 출력
- d: YOLOv5 결과 특정 부분 이미지 폴더 내용 삭제
- i: YOLOv5 실행 결과 좌표 값에 있는 이미지 추출
- c: 추출된 이미지 CNN 모델에 입력
    - 1: 컬러 이미지
    - 2: 회색조 이미지
        - 0per DB에 IP주소와 Count +

# 1. import 문

In [2]:
import cv2
from os import remove
from PIL import Image
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from gtts import gTTS
from playsound import playsound
import IPython.display as ipd

import pyodbc
import socket

# 2. DataBase 연결

In [3]:
# SQL Server 접속
server = 'SQL Server IP'
# Database 이름
database = 'DB name'
# 사용자 이름
username = 'user name'
# 비밀번호
password = 'password'

# 접속 코드
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password)
curosor = cnxn.cursor()

# 3. 현재 DB의 테이블 출력

In [4]:
curosor.execute("SELECT * FROM AddressTable_Count")
row_a = curosor.fetchone()
while row_a:
    print('DB 값: ', row_a.IP_Address, row_a.Count)
    row_a = curosor.fetchone()

DB 값:  192.168.35.126 3
DB 값:  192.168.37.174 2
DB 값:  시작 주소 0


# 4. 폴더 위치 설정

In [5]:
# 출력 Image
folder1 = "./runs/detect/exp1/Cameras.jpg"
floder2 = "./runs/detect/exp2/Cameras_gray.jpg"

# 저장 Image
file1 = './img/Cameras.jpg'
file2 = './img/Cameras_gray.jpg'
save_file1 = './img/cnn.jpg'
save_file2 = './img/cnn_gray.jpg'

# 바운딩 박스 위치
label1 = './runs/detect/exp1/labels/Cameras.txt'
label2 = './runs/detect/exp2/labels/Cameras_gray.txt'

# 6. 함수

In [6]:
# txt 파일 읽기에 사용
def im_trim(img, x, y, w, h):
    imgtrim = img[y: y + h, x: x + w]
    return imgtrim

# 5. 프로그램 실행

- q: 종료
- s: 이미지 저장
- y: YOLOv5 실행
- p: YOLOv5 결과 출력
- d: YOLOv5 결과 특정 부분 이미지 폴더 내용 삭제
- i: YOLOv5 실행 결과 좌표 값에 있는 이미지 추출
- c: 추출된 이미지 CNN 모델에 입력
    - 1: 컬러 이미지
    - 2: 회색조 이미지
        - 0per DB에 IP주소와 Count +
- b: 데이터베이스 조회
- z: 데이터베이스 연결 종료

In [7]:
# 카메라 연결
video_capture = cv2.VideoCapture(1)
print('Original width: %d, height: %d' % (video_capture.get(cv2.CAP_PROP_FRAME_WIDTH), video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)))

while (True):
    # 카메라 영상 받기
    ret, frame = video_capture.read()
    cv2.imshow('Original Video', frame)

    # 키보드 q 누르면 종료
    key = cv2.waitKey(1)
    if key == ord('q'):
        print('종료')
        break
    
    # 키보드 s 누르면 이미지 저장
    elif key == ord('s'):
        print('이미지 저장')

        cv2.imwrite(file1, frame)
        img = cv2.imread(file1)
        print(file1, ' saved')

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(file2, gray)
        print(file2, ' saved')

    # 키보드 y 누르면 YOLOv5 모델 실행
    elif key == ord('y'):
        print('YOLOv5 모델 실행 -> 이미지')
        
        !cd yolov5 && python detect.py --source ../img/Cameras.jpg --weights ../blocks_m_20.pt --img 640 --conf 0.3 --iou 0.5 --save-txt --project ../runs/detect --name exp1 --exist-ok
        print('컬러 이미지 YOLOv5 완료')

        !cd yolov5 && python detect.py --source ../img/Cameras_gray.jpg --weights ../blocks_m_20.pt --img 640 --conf 0.3 --iou 0.5 --save-txt --project ../runs/detect --name exp2 --exist-ok
        print('회색 이미지 YOLOv5 완료')

    # 키보드 0 누르면 실시간 YOLOv5 모델로 전환
    elif key == ord('0'):
        print('YOLOv5 모델 실행 -> 실시간 영상')
        !cd yolov5 && python detect.py --source 0 --weights ../blocks_m_20.pt --img 640 --conf 0.25 --iou 0.5 --save-txt --project ../runs/detect --name exp3 --exist-ok
        
    # 키보드 p 누르면 이미지 출력
    elif key == ord('p'):
        print('이미지 출력')

        img = Image.open(folder1)
        img.show()

        img = Image.open(floder2)
        img.show()

    # 키보드 d 누르면 폴더 내 이미지 삭제
    elif key == ord('d'):
        print('폴더 내 이미지 삭제')

        def DeleteAllFiles(filePath):
            if os.path.exists(filePath):
                for file in os.scandir(filePath):
                    os.remove(file.path)
                    print(filePath)
                    print(file.path)
                return 'Remove All File'
            else:
                return 'Directory Not Found'
                
        print(DeleteAllFiles('./img/cnn_img/'))

    # YOLOv5 실행 후 좌표값으로 이미지 추출
    elif key ==ord('i'):
        print('YOLOv5 실행 후 좌표값으로 이미지 추출')

        f = open(label1, 'r')
        lines = f.readlines()
        num1 = 0

        for line in lines:
            data = line.split('\n')
            data = data[0].split(' ')

            x1 = float(data[1])
            y1 = float(data[2])
            x2 = float(data[3])
            y2 = float(data[4])

            img = cv2.imread(file1)
            height, width, channel = img.shape

            dw = 1. / width
            dh = 1. / height
            x = x1 / dw
            y = y1 / dh
            w = round (x2 / dw)
            h = round (y2 / dh)
            x1 = round((2 * x - w) / 2)
            y1 =round((2 * y - h) / 2)

            result = im_trim(img, x1, y1, w, h)
            cv2.imwrite(f'./img/cnn_img/cnn{num1}.jpg', result)

            num1 += 1
        print('추출 컬러 이미지 개수: ', num1)

        f = open(label2, 'r')
        lines = f.readlines()
        num2 = 0
        
        # 라벨 파일이 없을 경우 생성
        with open(label1,'w') as f:
            pass

        for line in lines:
            data = line.split('\n')
            data = data[0].split(' ')

            x1 = float(data[1])
            y1 = float(data[2])
            x2 = float(data[3])
            y2 = float(data[4])
        
            img = cv2.imread(file2)
            height, width, channel = img.shape

            dw = 1. / width
            dh = 1. / height
            x = x1 / dw
            y = y1 / dh
            w = round (x2 / dw)
            h = round (y2 / dh)
            x1 = round((2 * x - w) / 2)
            y1 =round((2 * y - h) / 2)

            result = im_trim(img, x1, y1, w, h)
            cv2.imwrite(f'./img/cnn_img/cnn_gray{num2}.jpg', result)
            num2 += 1

        with open(label2,'w') as f:
            pass

        print('추출 회색조 이미지 개수: ', num2)
    
    # 키보드 c 누르면 CNN 모델 실행
    elif key == ord('c'):
        print('C Key')
        count = 0
        # CNN 모델 불러오기
        model = load_model('./model.h5')
        number = input(print('어떤 이미지를 불러올까요? [1: 컬러 이미지, 2: 회색조 이미지]'))

        if number == '1':
            print('1 Key')

            # 컬러 이미지 불러오기 및 전처리
            for i in range(num1):
                print(num1)
                img = cv2.imread(f'./img/cnn_img/cnn{i}.jpg', cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (128, 128))
                img = img.reshape(1, 128, 128, 1)
                img = img / 255.0
            
                predict = model.predict(img)
                #print(predict)
                #print(np.argmax(predict))
                if np.argmax(predict) == 0:
                    print(f'cnn{i}.jpg 회손율 = 0%')
                    os.system('0per.wav')
                    #ipd.Audio('0per.wav')
                elif np.argmax(predict) == 1:
                    print(f'cnn{i}.jpg 회손율 = 25%')
                    os.system('25per.wav')
                    #ipd.Audio('25per.wav')
                    count = 1
                elif np.argmax(predict) == 2:
                    print(f'cnn{i}.jpg 회손율 = 50%')
                    os.system('50per.wav')
                    #ipd.Audio('50per.wav')
                    count = 1
                elif np.argmax(predict) == 3:
                    print(f'cnn{i}.jpg 회손율 = 75%')
                    os.system('75per.wav')
                    #ipd.Audio('75per.wav')
                    count = 1
        
        elif number == '2':
            print('2 Key')

            # 회색조 이미지 불러오기 및 전처리
            for i in range(num2):
                print(num2)
                img = cv2.imread(f'./img/cnn_img/cnn_gray{i}.jpg', cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (128, 128))
                img = img.reshape(1, 128, 128, 1)
                img = img / 255.0
            
                predict = model.predict(img)
                #print(predict)
                #print(np.argmax(predict))
                if np.argmax(predict) == 0:
                    print(f'cnn_gray{i}.jpg 회손율 = 0%')
                    os.system('0per.wav')
                    #ipd.Audio('0per.wav')
                elif np.argmax(predict) == 1:
                    print(f'cnn_gray{i}.jpg 회손율 = 25%')
                    os.system('25per.wav')
                    #ipd.Audio('25per.wav')
                    count = 1
                elif np.argmax(predict) == 2:
                    print(f'cnn_gray{i}.jpg 회손율 = 50%')
                    os.system('50per.wav')
                    #ipd.Audio('50per.wav')
                    count = 1
                elif np.argmax(predict) == 3:
                    print(f'cnn_gray{i}.jpg 회손율 = 75%')
                    os.system('75per.wav')
                    #ipd.Audio('75per.wav')
                    count = 1

        if count == 1:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.connect(("pwnbit.kr", 443))
            IP = sock.getsockname()[0]
            print(IP)
            
            curosor.execute("SELECT * FROM AddressTable_Count WHERE IP_Address= ?", IP)
            row = curosor.fetchall()
            if row == []:
                num=0
                print("없음")
            else:
                num=1
                print("있음")
                curosor.execute("UPDATE AddressTable_Count SET Count = ? WHERE IP_Address = ?", row[0][1]+1, IP)
                cnxn.commit()
                        


            print(num)
            curosor.execute("SELECT * FROM AddressTable_Count")
            row = curosor.fetchone()

            while row:
                # Db에 현재 IP 주소 값이 없을 경우 IP 주소 추가
                if num == 0:
                    print('생성')
                    curosor.execute("insert into AddressTable_Count (IP_Address, Count) VALUES (?, ?)", IP, 1)
                    cnxn.commit()
                break
                

    # 키보드 b 누르면 DB 출력
    elif key == ord('b'):
        print('B Key')

        curosor.execute("SELECT * FROM AddressTable_Count")
        row_a = curosor.fetchone()
        while row_a:
            print('DB 값: ', row_a.IP_Address, row_a.Count)
            row_a = curosor.fetchone()

    elif key == ord('z'):
        # SQL Server 접속 종료
        cnxn.close()
        print('DB 종료')

video_capture.release()
cv2.destroyAllWindows()

Original width: 640, height: 480
YOLOv5 모델 실행 -> 이미지
컬러 이미지 YOLOv5 완료


detect: weights=['../blocks_m_20.pt'], source=../img/Cameras.jpg, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.3, iou_thres=0.5, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../runs/detect, name=exp1, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  2023-6-18 Python-3.7.16 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 212 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
image 1/1 C:\Users\tjddb\Desktop\Program\jupyter\project\Block\img\Cameras.jpg: 480x640 1 Braille-block, 1 Braille-block-Destruction, 974.4ms
Speed: 2.0ms pre-process, 974.4ms inference, 26.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to ..\runs\detect\exp1
1 labels saved to ..\runs\detect\exp1\labels
detect: weights=['../blocks_m_20.pt'], source=../img/Cameras_gray.jpg, data=da

회색 이미지 YOLOv5 완료
YOLOv5 실행 후 좌표값으로 이미지 추출
추출 컬러 이미지 개수:  2
추출 회색조 이미지 개수:  2
C Key
어떤 이미지를 불러올까요? [1: 컬러 이미지, 2: 회색조 이미지]
1 Key
2
1/1 [==============================] - 0s 359ms/step
cnn0.jpg 회손율 = 75%
2
1/1 [==============================] - 0s 43ms/step
cnn1.jpg 회손율 = 75%
192.168.35.126
있음
1
B Key
DB 값:  192.168.35.126 4
DB 값:  192.168.37.174 2
DB 값:  시작 주소 0
B Key
DB 값:  192.168.35.126 4
DB 값:  192.168.37.174 2
DB 값:  시작 주소 0
DB 종료
종료
